1. Setup Environment 🔽

This cell downloads the initial setup.py script from your fork and then executes it, instructing the setup process to pull all subsequent project files from your repository.

In [1]:
import os
from pathlib import Path

# --- Start of Directory Management ---
BASE_DIR = Path("/teamspace/studios/this_studio")
if os.getcwd() != str(BASE_DIR):
    print(f"🔄 Changing directory from {os.getcwd()} to {BASE_DIR}")
    os.chdir(BASE_DIR)
# --- End of Directory Management ---

# @title Setup Environment.

# Set language and branch. You can change 'en' to 'ru' for Russian.
lang, branch = 'en', 'main'

# Define the local directory where scripts will be stored
scripts_dir = BASE_DIR / 'ANXETY' / 'scripts'
setup_script_path = scripts_dir / 'setup.py'

# Ensure the local script directory exists
os.makedirs(scripts_dir, exist_ok=True)

# Download the setup.py script directly from YOUR forked GitHub repository
setup_url = f'https://raw.githubusercontent.com/drf0rk/sdAIgenLightning/{branch}/scripts/setup.py'
!curl -sLo {setup_script_path} {setup_url}

# Run the downloaded setup script to get core files and initial settings
%run {setup_script_path} --lang=$lang --branch=$branch --fork=drf0rk/sdAIgenLightning

# --- Post-setup.py: Ensure critical modules are loaded and paths are set --- 
import sys
if str(BASE_DIR / 'ANXETY' / 'modules') not in sys.path:
    sys.path.insert(0, str(BASE_DIR / 'ANXETY' / 'modules'))
if str(BASE_DIR / 'ANXETY' / 'scripts') not in sys.path:
    sys.path.insert(0, str(BASE_DIR / 'ANXETY' / 'scripts'))

import json_utils as js
from webui_utils import _set_webui_paths, SHARED_MODEL_BASE # Import specific utilities

# Force update WEBUI paths using the UI selected in settings.json (or default 'A1111')
# This ensures `model_dir`, `vae_dir`, etc., are correctly pointing to SHARED_MODEL_BASE
current_ui_selection = js.read(BASE_DIR / 'ANXETY' / 'settings.json', 'WEBUI.current', 'A1111')
print(f"🔄 Ensuring WebUI paths are configured for {current_ui_selection}...")
_set_webui_paths(current_ui_selection)

# Re-load settings after forcing _set_webui_paths to ensure the latest paths are used.
settings = js.read(BASE_DIR / 'ANXETY' / 'settings.json')

# Define global path variables explicitly in the notebook's scope from settings
# These will be used by subsequent cells, ensuring consistency.
WEBUI_PATH = Path(settings.get('WEBUI', {}).get('webui_path', str(BASE_DIR / current_ui_selection)))
model_dir = Path(settings.get('WEBUI', {}).get('model_dir', str(SHARED_MODEL_BASE / 'Stable-diffusion')))
vae_dir = Path(settings.get('WEBUI', {}).get('vae_dir', str(SHARED_MODEL_BASE / 'vae')))
lora_dir = Path(settings.get('WEBUI', {}).get('lora_dir', str(SHARED_MODEL_BASE / 'Lora')))
embed_dir = Path(settings.get('WEBUI', {}).get('embed_dir', str(SHARED_MODEL_BASE / 'embeddings')))
control_dir = Path(settings.get('WEBUI', {}).get('control_dir', str(SHARED_MODEL_BASE / 'ControlNet')))
adetailer_dir = Path(settings.get('WEBUI', {}).get('adetailer_dir', str(SHARED_MODEL_BASE / 'adetailer')))
extension_dir = Path(settings.get('WEBUI', {}).get('extension_dir', str(WEBUI_PATH / 'extensions')))

# --- Ensure we are back in BASE_DIR at the end of the cell ---
if os.getcwd() != str(BASE_DIR):
    os.chdir(BASE_DIR)
# --- End of Directory Management ---


🔄 Ensuring WebUI paths are configured for A1111...


2. Widgets 🔽

This cell will load the interactive widgets. Since setup.py (executed in the previous cell) has already downloaded the patched widgets-en.py (or widgets-ru.py) to scripts_dir, this command will now use your modified version with the LoRA dropdown.

In [2]:
import os
from pathlib import Path

# --- Start of Directory Management ---
BASE_DIR = Path("/teamspace/studios/this_studio")
if os.getcwd() != str(BASE_DIR):
    print(f"🔄 Changing directory from {os.getcwd()} to {BASE_DIR}")
    os.chdir(BASE_DIR)
# --- End of Directory Management ---

# Define scripts_dir using BASE_DIR for consistency
scripts_dir = BASE_DIR / 'ANXETY' / 'scripts'
lang = 'en' # Ensure lang is defined, typically from the first cell

# Model selection, vae, control-net and more.
%run {scripts_dir}/{lang}/widgets-{lang}.py

# --- Ensure we are back in BASE_DIR at the end of the cell ---
if os.getcwd() != str(BASE_DIR):
    os.chdir(BASE_DIR)
# --- End of Directory Management ---


3. Downloading 🔽

This cell initiates the downloading process for models, VAEs, extensions, and other necessary components. It will use the patched downloading-en.py (or downloading-ru.py) from your fork, ensuring downloads go to your centralized model storage location.

In [3]:
import os
from pathlib import Path
import json_utils as js # Import json_utils to read settings
# This import needs to happen after sys.path is set,
# so we'll assume it's set by the first cell's execution,
# or we can add it here explicitly if needed.
# For simplicity, if json_utils isn't found here, try to load its path first.

# --- Start of Directory Management ---
BASE_DIR = Path("/teamspace/studios/this_studio")
if os.getcwd() != str(BASE_DIR):
    print(f"🔄 Changing directory from {os.getcwd()} to {BASE_DIR}")
    os.chdir(BASE_DIR)
# --- End of Directory Management ---

# Define scripts_dir and lang using BASE_DIR for consistency
scripts_dir = BASE_DIR / 'ANXETY' / 'scripts'
lang = 'en' # Ensure lang is defined, typically from the first cell

# --- Temporary Code for DEBUGging in Notebook Cell ---
# Load settings from settings.json to get the current widget values
# This will reflect what was saved by the Widgets cell
SETTINGS_PATH = BASE_DIR / 'ANXETY' / 'settings.json'
try:
    # Ensure modules path is set if not already from the setup cell
    anxety_modules_path = BASE_DIR / 'ANXETY' / 'modules'
    if str(anxety_modules_path) not in sys.path:
        sys.path.insert(0, str(anxety_modules_path))
    import json_utils as js
    settings_data = js.read(SETTINGS_PATH)
    model_selections = settings_data.get('WIDGETS', {}).get('model', ('none',))
    vae_selections = settings_data.get('WIDGETS', {}).get('vae', ('none',))
except ImportError:
    print("Warning: Could not import json_utils. Debugging info might be incomplete.")
    model_selections = ("Error: json_utils missing",)
    vae_selections = ("Error: json_utils missing",)
except Exception as e:
    print(f"Warning: Could not read settings for debugging: {e}")
    model_selections = ("Error reading settings",)
    vae_selections = ("Error reading settings",)

print('📦 Processing download selections...')
print(f"DEBUG: Selected Models: {model_selections}")
print(f"DEBUG: Selected VAEs: {vae_selections}")
# --- End of Temporary Code for DEBUGging ---

# Downloading libraries, repos, models and more.
# The downloading script will now correctly use the global model_dir, vae_dir, etc.,
# which are already defined in the first cell based on SHARED_MODEL_BASE.
%run {scripts_dir}/{lang}/downloading-{lang}.py

# --- Ensure we are back in BASE_DIR at the end of the cell ---
if os.getcwd() != str(BASE_DIR):
    os.chdir(BASE_DIR)
# --- End of Directory Management ---

📦 Processing download selections...
DEBUG: Selected Models: ['1. D5K6.0']
DEBUG: Selected VAEs: ['1. vae-ft-mse-840000-ema-pruned | 840000 | 840k SD1.5 VAE - vae-ft-mse-840k']
✨ Downloader Version: 2025.06.09.8_final_patch
📦 Processing download selections...
Starting downloads...
【#61abfb 1.7GiB/1.9GiB(90%) CN:16 DL:284MiB】                                                                                  
61abfb|OK  |   283MiB/s|/teamspace/studios/this_studio/sd_models_shared/Stable-diffusion/D5K6.0.safetensors
【#716d1f 169MiB/319MiB(53%) CN:16 DL:206MiB】                                                                                  
716d1f|OK  |   229MiB/s|/teamspace/studios/this_studio/sd_models_shared/vae/vaeFtMse840000EmaPruned_vaeFtMse840k.safetensors
【#bf6161 297MiB/297MiB(99%) CN:2 DL:205MiB】                                                                                   
bf6161|OK  |   204MiB/s|/teamspace/studios/this_studio/sd_models_shared/Lora/concept-female_masturbation-

4. Start 🔽

This cell launches the Stable Diffusion WebUI. It will execute the patched launch.py script from your fork, applying any platform-specific optimizations and arguments you've included.

In [ ]:
import os
from pathlib import Path

# --- Start of Directory Management ---
BASE_DIR = Path("/teamspace/studios/this_studio")
if os.getcwd() != str(BASE_DIR):
    print(f"🔄 Changing directory from {os.getcwd()} to {BASE_DIR}")
    os.chdir(BASE_DIR)
# --- End of Directory Management ---

# Define scripts_dir using BASE_DIR for consistency
scripts_dir = BASE_DIR / 'ANXETY' / 'scripts'

# Launch WebUI.
%run {scripts_dir}/launch.py -l

# --- Ensure we are back in BASE_DIR at the end of the cell ---
if os.getcwd() != str(BASE_DIR):
    os.chdir(BASE_DIR)
# --- End of Directory Management ---


Utilities
5. Run Cleanup Utility 🔽

This cell runs the new Cleanup Utility GUI script, allowing you to manage your environment. This script is downloaded from your fork via the initial setup.

In [ ]:
import os
from pathlib import Path

# --- Start of Directory Management ---
BASE_DIR = Path("/teamspace/studios/this_studio")
if os.getcwd() != str(BASE_DIR):
    print(f"🔄 Changing directory from {os.getcwd()} to {BASE_DIR}")
    os.chdir(BASE_DIR)
# --- End of Directory Management ---

# Define scripts_dir using BASE_DIR for consistency
scripts_dir = BASE_DIR / 'ANXETY' / 'scripts'

# Run the Cleanup Utility GUI
%run {scripts_dir}/auto-cleaner.py

# --- Ensure we are back in BASE_DIR at the end of the cell ---
if os.getcwd() != str(BASE_DIR):
    os.chdir(BASE_DIR)
# --- End of Directory Management ---


In [ ]:
from pathlib import Path
import shutil
import os
import sys

# --- Start of Directory Management ---
BASE_DIR = Path("/teamspace/studios/this_studio")
if os.getcwd() != str(BASE_DIR):
    print(f"🔄 Changing directory from {os.getcwd()} to {BASE_DIR}")
    os.chdir(BASE_DIR)
# --- End of Directory Management ---

# --- VERY IMPORTANT WARNING ---
# This cell will DELETE almost everything in your Lightning AI instance.
# It will only preserve this notebook file and 'main.py'.
# Ensure you understand what is being deleted before running.
# This operation is IRREVERSIBLE.

print("!!! DANGER: YOU ARE ABOUT TO DELETE ALMOST ALL FILES !!!")
print("!!! PLEASE READ CAREFULLY BEFORE PROCEEDING !!!")
print("\nThis cell will delete all folders and files in your current studio instance, EXCEPT:")
print(" - This notebook file (e.g., 'LightningAnxiety (1) (1) (2).ipynb')")
print(" - The 'main.py' file (if it exists in the root)")
print("\nTHIS WILL REQUIRE YOU TO RERUN THE ENTIRE NOTEBOOK FROM THE FIRST CELL FOR A FRESH START.")
print("If you have any custom files you wish to keep, MOVE THEM OUTSIDE THIS STUDIO INSTANCE NOW.")
print("Proceed only if you want a completely blank studio environment.")
print("\nType 'YES_DELETE_ALL' (case-sensitive) to confirm deletion and execute, then press Enter.")
print("Anything else will abort the operation.")

confirmation = input("Confirmation: ")

if confirmation.strip() == "YES_DELETE_ALL": # Case-sensitive comparison
    # Get the home/studio path
    # On Lightning AI, this is typically /teamspace/studios/this_studio
    HOME_PATH = Path.home()

    # Get the current notebook's filename
    # Use a more robust way to find the notebook file name, as __file__ is not always reliable in notebooks
    notebook_filename = "LightningAnxiety (1) (1) (2).ipynb" # Explicitly set your notebook filename here for reliability
    notebook_path = HOME_PATH / notebook_filename

    main_py_path = HOME_PATH / "main.py"

    # Define items to EXCLUDE from deletion
    EXCLUDE_LIST = [
        notebook_path,
        main_py_path
    ]

    print(f"\n--- Starting Comprehensive Deletion in {HOME_PATH} ---")
    deleted_count = 0
    skipped_count = 0

    for item in HOME_PATH.iterdir():
        # Convert item to absolute path for consistent comparison with EXCLUDE_LIST
        abs_item = item.resolve()
        if abs_item in EXCLUDE_LIST:
            print(f"ℹ️ Skipping protected item: {item.name}")
            skipped_count += 1
            continue

        print(f"🗑️ Attempting to delete: {item.name} ({item})...")
        try:
            if item.is_dir():
                shutil.rmtree(item)
            else:
                item.unlink() # Delete file
            print(f"✅ Successfully deleted: {item.name}")
            deleted_count += 1
        except Exception as e:
            print(f"❌ Error deleting {item.name} ({item}): {e}")

    print("\n--- Comprehensive Cleanup Process Complete ---")
    print(f"Summary: {deleted_count} items deleted, {skipped_count} items skipped (protected).")
    print("Please restart your runtime and run the notebook from the first cell for a fresh start.")
else:
    print("\nOperation aborted. No folders were deleted.\n")

# --- Ensure we are back in BASE_DIR at the end of the cell ---
if os.getcwd() != str(BASE_DIR):
    os.chdir(BASE_DIR)
# --- End of Directory Management ---
